<a href="https://colab.research.google.com/github/barrettbeatrice/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#OPTIMIZATION ATTEMPT 1, 2, 3 at 75% Accuracy



In [74]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()



,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [75]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [76]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [77]:
# Look at APPLICATION_TYPE value counts for binning
type_counts = application_df["APPLICATION_TYPE"].value_counts()

In [78]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(type_counts[type_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [79]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()
print(class_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [80]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts_gt1 = class_counts.loc[class_counts > 1]
class_counts_gt1.head()


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Name: count, dtype: int64

In [81]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [82]:
# Convert categorical data to numeric with `pd.get_dummies`
application_numeric = pd.get_dummies(application_df)

In [83]:
# Split our preprocessed data into our features and target arrays
X = application_numeric.drop(['IS_SUCCESSFUL'], axis=1)
y = application_numeric['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [84]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_train_scaled.shape

(25724, 43)

**Compile, Train and Evaluate the Model**

In [85]:
# Define the model - Adding an additional hidden layer, with less nodes, to try to increase accuracy.
# Increasing Epochs to 150
# Adding another hidden layer
# Increasing input features


number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 15

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,
             input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer2, activation="relu"))


# Third hidden layer
nn.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 80)                3520      
                                                                 
 dense_27 (Dense)            (None, 30)                2430      
                                                                 
 dense_28 (Dense)            (None, 15)                465       
                                                                 
 dense_29 (Dense)            (None, 1)                 16        
                                                                 
Total params: 6431 (25.12 KB)
Trainable params: 6431 (25.12 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [86]:
# Compile the model
#Using 'adam' again, as it is a generally widely used optimizer metric
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [87]:
# Train the model, add more epochs, to 150
fit_model = nn.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 3s 2ms/step - loss: 0.7049 - accuracy: 0.7088
Epoch 2/150
804/804 [==============================] - 1s 2ms/step - loss: 0.6449 - accuracy: 0.7211
Epoch 3/150
804/804 [==============================] - 1s 2ms/step - loss: 0.6533 - accuracy: 0.7225
Epoch 4/150
804/804 [==============================] - 1s 2ms/step - loss: 0.8473 - accuracy: 0.7148
Epoch 5/150
804/804 [==============================] - 1s 2ms/step - loss: 0.6249 - accuracy: 0.7181
Epoch 6/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5791 - accuracy: 0.7252
Epoch 7/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5902 - accuracy: 0.7098
Epoch 8/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5673 - accuracy: 0.7243
Epoch 9/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5711 - accuracy: 0.7273
Epoch 10/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5705 - accura

In [88]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6086 - accuracy: 0.7214 - 466ms/epoch - 2ms/step
Loss: 0.6085622906684875, Accuracy: 0.7213994264602661


In [89]:
#Accuracy actually decreased

Definitely crossed the bridge by adding too many epochs and resulting in some over-fitting in the model training. Trying again.

**Optimization Attempt 2**


In [90]:
#This time: Let's add a fourth hidden layer, with less units, and decrease the epochs to 50

In [91]:
# Reread csv
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [92]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN','NAME'])

In [93]:
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [94]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# Also am going to try decreasing the cutoff point to allow for more data input
application_types_to_replace = list(type_counts[type_counts < 100].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64

In [95]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()
print(class_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [96]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# As above, also lower classification cutoff to have mroe bins
classifications_to_replace = list(class_counts[class_counts < 500].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [97]:
# Convert categorical data to numeric with `pd.get_dummies`
application_numeric = pd.get_dummies(application_df)

In [98]:
# Split our preprocessed data into our features and target arrays
X = application_numeric.drop(['IS_SUCCESSFUL'], axis=1)
y = application_numeric['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [99]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_train_scaled.shape

(25724, 45)

In [100]:
# Define the model - Add a fourth hidden layer

number_input_features_3 = len(X_train_scaled[0])
hidden_nodes_layer1_3 = 80
hidden_nodes_layer2_3 = 30
hidden_nodes_layer3_3 = 10
hidden_nodes_layer4_3 = 5


nn_change3 = tf.keras.models.Sequential()

# First hidden layer
nn_change3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1_3,
             input_dim=number_input_features_3, activation="tanh"))

# Second hidden layer
nn_change3.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer2_3, activation="relu"))

# Third hidden layer
nn_change3.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer3_3, activation="relu"))

#Fourth hidden layer
nn_change3.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer4_3, activation="relu"))

# Output layer
nn_change3.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn_change3.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 80)                3680      
                                                                 
 dense_31 (Dense)            (None, 30)                2430      
                                                                 
 dense_32 (Dense)            (None, 10)                310       
                                                                 
 dense_33 (Dense)            (None, 5)                 55        
                                                                 
 dense_34 (Dense)            (None, 1)                 6         
                                                                 
Total params: 6481 (25.32 KB)
Trainable params: 6481 (25.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [101]:
# Compile the model
nn_change3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [102]:
# Train the model
fit_model3 = nn_change3.fit(X_train_scaled, y_train, epochs=75)

Epoch 1/75
804/804 [==============================] - 4s 3ms/step - loss: 0.6458 - accuracy: 0.6995
Epoch 2/75
804/804 [==============================] - 2s 2ms/step - loss: 0.6002 - accuracy: 0.7047
Epoch 3/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5808 - accuracy: 0.7237
Epoch 4/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5788 - accuracy: 0.7267
Epoch 5/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5682 - accuracy: 0.7264
Epoch 6/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5651 - accuracy: 0.7285
Epoch 7/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5648 - accuracy: 0.7275
Epoch 8/75
804/804 [==============================] - 2s 3ms/step - loss: 0.5740 - accuracy: 0.7184
Epoch 9/75
804/804 [==============================] - 2s 3ms/step - loss: 0.5656 - accuracy: 0.7259
Epoch 10/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5674 - accuracy: 0.7268

In [103]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_change3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5620 - accuracy: 0.7268 - 455ms/epoch - 2ms/step
Loss: 0.5620163679122925, Accuracy: 0.7267638444900513


In [104]:
# Export our model to HDF5 file
nn_change3.save('AlphabetSoupCharity_Optimization.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Still not able to achieve 75% accuracy, after changing around various metrics of the model for a second time. Trying again, focusing on keeping 3 hidden layers and making the units for nodes a lot smaller and simpler.

**Attempt at Optimization 3**

In [105]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features_4 = len(X_train_scaled[0])
hidden_nodes_layer1_4 = 10
hidden_nodes_layer2_4 = 8
hidden_nodes_layer3_4 = 6

nn_change4 = tf.keras.models.Sequential()

# First hidden layer
nn_change4.add(tf.keras.layers.Dense(units=hidden_nodes_layer1_4,
             input_dim=number_input_features_4, activation="relu"))


# Second hidden layer
nn_change4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_4,
             input_dim=number_input_features_4, activation="relu"))

# Third hidden layer
nn_change4.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer3_4, activation="relu"))

# Output layer
nn_change4.add(tf.keras.layers.Dense(units=1, activation="relu"))


# Check the structure of the model
nn_change4.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_35 (Dense)            (None, 10)                460       
                                                                 
 dense_36 (Dense)            (None, 8)                 88        
                                                                 
 dense_37 (Dense)            (None, 6)                 54        
                                                                 
 dense_38 (Dense)            (None, 1)                 7         
                                                                 
Total params: 609 (2.38 KB)
Trainable params: 609 (2.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [106]:
# Compile the model
nn_change4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [107]:
# Train the model
fit_model4 = nn_change4.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 1.4774 - accuracy: 0.5791
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.7158 - accuracy: 0.6763
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6795 - accuracy: 0.7053
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6480 - accuracy: 0.7153
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6377 - accuracy: 0.7204
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6297 - accuracy: 0.7224
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6235 - accuracy: 0.7242
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6189 - accuracy: 0.7259
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6145 - accuracy: 0.7268
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5973 - accura

In [108]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_change4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5690 - accuracy: 0.7285 - 452ms/epoch - 2ms/step
Loss: 0.5689913630485535, Accuracy: 0.7285131216049194


In [109]:
# Save the model to a HDF5 file
nn_change4.save('AlphabetSoupCharity_Optimization2.h5')